In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
import time

# Load YOLOv8s model
model = YOLO("runs/detect/train6/weights/best.pt")

# Input video paths
video_paths = ['videos/video1.mp4', 'videos/video2.mp4', 'videos/video3.mp4', 'videos/video4.mp4']
caps = [cv2.VideoCapture(p) for p in video_paths]

# Output video config
SUB_W, SUB_H = 640, 480
out_w, out_h = 2 * SUB_W, 2 * SUB_H
fps = int(caps[0].get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output_grid_with_green.mp4", fourcc, fps, (out_w, out_h))

# Define vehicle weights
class_weights = {
    0: 1.0,   # car
    1: 3.0,   # motorcycle
    2: 4.0,   # bus
    3: 2.5,   # truck
    4: 5.0    # auto
}

# Green time calculator
def compute_green_time(counts, speeds):
    total_score = 0
    total_weight = 0
    for cls_id, count in counts.items():
        weight = class_weights.get(cls_id, 1.0)
        avg_speed = speeds[cls_id] / max(1, counts[cls_id])  # avoid division by zero
        # Factor: more vehicles & slower speed → longer green
        factor = weight * count * (1.5 - min(avg_speed / 60, 1.0))  # normalize speed influence
        total_score += factor
        total_weight += weight * count
    if total_weight == 0:
        return 10  # minimum green time
    time_alloc = np.clip(int(10 + (total_score / total_weight) * 5), 10, 60)
    return time_alloc

# Process video
frame_idx = 0
tracker = [None] * 4
counts_list = [defaultdict(int) for _ in range(4)]
speeds_list = [defaultdict(float) for _ in range(4)]

while True:
    frames = []
    for cap in caps:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(cv2.resize(frame, (SUB_W, SUB_H)))
    if len(frames) < 4:
        break

    dets = model(frames, imgsz=640)
    disp_frames = []

    for idx in range(4):
        sub = frames[idx].copy()
        count_dict = counts_list[idx]
        speed_dict = speeds_list[idx]
        det = dets[idx]

        speeds = []
        if det.boxes is not None:
            for box, cls, conf in zip(det.boxes.xyxy, det.boxes.cls, det.boxes.conf):
                x1, y1, x2, y2 = map(int, box)
                cls_id = int(cls)
                w, h = x2 - x1, y2 - y1
                count_dict[cls_id] += 1
                pixel_to_meter_ratio = 0.05  # ← adjust this
                speed = ((w + h) / 2) * pixel_to_meter_ratio
                speed_dict[cls_id] += speed
                color = (0, 255, 0)
                cv2.rectangle(sub, (x1, y1), (x2, y2), color, 2)
                cv2.putText(sub, f"{model.names[cls_id]} {conf:.2f}", (x1, y1 - 6),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Display stats
        y = 20
        for cid in sorted(count_dict.keys()):
            avg_spd = speed_dict[cid] / max(1, count_dict[cid])
            cv2.putText(sub, f"{model.names[cid]}: {count_dict[cid]}, avg_spd={avg_spd:.1f}",
                        (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            y += 20

        # Compute and show green time
        green_time = compute_green_time(count_dict, speed_dict)
        cv2.putText(sub, f"Green Time: {green_time}s", (10, y + 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

        disp_frames.append(sub)

        # Reset for next computation window
        if frame_idx % (fps * 15) == 0:  # recompute every 15s
            counts_list[idx] = defaultdict(int)
            speeds_list[idx] = defaultdict(float)

    # Arrange 2x2 grid
    top = np.hstack(disp_frames[:2])
    bottom = np.hstack(disp_frames[2:])
    grid = np.vstack([top, bottom])

    out.write(grid)

    cv2.imshow("Traffic Monitoring", grid)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_idx += 1

# Cleanup
for cap in caps:
    cap.release()
out.release()
cv2.destroyAllWindows()



0: 480x640 6 cars, 2 motorcycles, 1 truck, 20.9ms
1: 480x640 4 cars, 5 motorcycles, 20.9ms
2: 480x640 11 cars, 4 motorcycles, 2 trucks, 1 auto, 20.9ms
3: 480x640 3 cars, 2 motorcycles, 1 bus, 3 trucks, 20.9ms
Speed: 3.6ms preprocess, 20.9ms inference, 40.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 cars, 3 motorcycles, 2 trucks, 18.0ms
1: 480x640 3 cars, 4 motorcycles, 18.0ms
2: 480x640 10 cars, 3 motorcycles, 2 trucks, 1 auto, 18.0ms
3: 480x640 4 cars, 3 motorcycles, 1 bus, 3 trucks, 1 auto, 18.0ms
Speed: 2.7ms preprocess, 18.0ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 cars, 4 motorcycles, 2 trucks, 18.0ms
1: 480x640 3 cars, 4 motorcycles, 2 trucks, 18.0ms
2: 480x640 9 cars, 4 motorcycles, 2 trucks, 18.0ms
3: 480x640 4 cars, 3 motorcycles, 2 trucks, 1 auto, 18.0ms
Speed: 2.7ms preprocess, 18.0ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 cars, 3 motorcycles, 1 truck, 17.1ms
1: 480x640 5 ca

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
import pandas as pd
import time

# ─── CONFIGURATION ──────────────────────────────────────────────
video_paths = [
    "videos/video1.mp4",
    "videos/video2.mp4",
    "videos/video3.mp4",
    "videos/video4.mp4"
]
model_path = "runs/detect/train4/weights/best.pt"
meters_per_pixel = 0.05  # Adjust based on your camera calibration
INFER_IMG_SIZE = 640

font = cv2.FONT_HERSHEY_SIMPLEX
colors = {
    0: (0, 255, 0),     # car
    1: (0, 0, 255),     # motorcycle
    2: (255, 0, 0),     # bus
    3: (0, 255, 255),   # truck
    4: (255, 0, 255)    # auto
}
class_map = {
    0: "car",
    1: "motorcycle",
    2: "bus",
    3: "truck",
    4: "auto"
}
# ───────────────────────────────────────────────────────────────────

# ─── INITIALIZATION ────────────────────────────────────────────────
caps = [cv2.VideoCapture(p) for p in video_paths]
assert all(c.isOpened() for c in caps), "One or more videos failed to open."

model = YOLO(model_path)
fps = int(caps[0].get(cv2.CAP_PROP_FPS))
frame_w = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))

SUB_W = 480
SUB_H = int(frame_h * (SUB_W / frame_w))

out_w, out_h = SUB_W * 2, SUB_H * 2
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output_grid.mp4", fourcc, fps, (out_w, out_h))

cv2.namedWindow("Multi-Video Analysis", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Multi-Video Analysis", out_w, out_h)

unique_ids = [set() for _ in range(4)]
last_centroids = [dict() for _ in range(4)]
speed_samples = [list() for _ in range(4)]
classwise_counts = [defaultdict(int) for _ in range(4)]
# ───────────────────────────────────────────────────────────────────

# ─── MAIN PROCESSING LOOP ──────────────────────────────────────────
while True:
    frames = []
    rets = []
    for cap in caps:
        ret, frame = cap.read()
        rets.append(ret)
        frames.append(frame if ret else np.zeros((frame_h, frame_w, 3), dtype=np.uint8))
    if not any(rets):
        break  # exit if all videos ended

    results = model.track(
        frames,
        imgsz=INFER_IMG_SIZE,
        conf=0.25,
        iou=0.45,
        persist=True
    )
    processed_subframes = []

    for idx, det in enumerate(results):
        sub = cv2.resize(frames[idx], (SUB_W, SUB_H))
        if det.boxes is not None and len(det.boxes) > 0:
            xyxy = det.boxes.xyxy.cpu().numpy()               # (N,4)
            cls_ids = det.boxes.cls.cpu().numpy().astype(int) # (N,)
            track_ids = det.boxes.id.cpu().numpy().astype(int) if det.boxes.id is not None else [-1] * len(cls_ids)

            for (x1, y1, x2, y2), class_idx, track_id in zip(xyxy, cls_ids, track_ids):
                class_idx = class_idx if class_idx in class_map else 4  # remap unknown to 'auto'
                cx = int((x1 + x2) / 2)
                cy = int((y1 + y2) / 2)

                scale_x = SUB_W / frame_w
                scale_y = SUB_H / frame_h
                sx1 = int(x1 * scale_x)
                sy1 = int(y1 * scale_y)
                sx2 = int(x2 * scale_x)
                sy2 = int(y2 * scale_y)
                scx = int(cx * scale_x)
                scy = int(cy * scale_y)

                color = colors[class_idx]
                cv2.rectangle(sub, (sx1, sy1), (sx2, sy2), color, 2)
                cv2.putText(sub, f"{track_id}:{class_idx}", (sx1, sy1 - 5), font, 0.5, color, 1)

                unique_ids[idx].add(track_id)
                classwise_counts[idx][class_idx] += 1

                prev = last_centroids[idx].get(track_id, None)
                if prev is not None:
                    dx = cx - prev[0]
                    dy = cy - prev[1]
                    pixel_dist = np.hypot(dx, dy)
                    speed_m_s = pixel_dist * fps * meters_per_pixel
                    speed_samples[idx].append(speed_m_s)
                    cv2.putText(sub, f"{speed_m_s:.1f} m/s", (sx1, sy2 + 12), font, 0.5, color, 1)

                last_centroids[idx][track_id] = (cx, cy)

        count = len(unique_ids[idx])
        avg_speed = np.mean(speed_samples[idx]) if speed_samples[idx] else 0
        green_time = max(10, min(60, int((count / (avg_speed + 1)) * 5)))  # simple heuristic
        stats = [
            f"Video {idx+1}",
            f"Count: {count}",
            f"Avg Speed: {avg_speed:.1f} m/s",
            f"Green Time: {green_time} s"
        ]
        y_offset = 20
        for stat in stats:
            cv2.putText(sub, stat, (10, y_offset), font, 0.6, (0, 0, 0), 3)
            cv2.putText(sub, stat, (10, y_offset), font, 0.6, (255, 255, 255), 1)
            y_offset += 25

        processed_subframes.append(sub)

    top = np.hstack((processed_subframes[0], processed_subframes[1]))
    bottom = np.hstack((processed_subframes[2], processed_subframes[3]))
    grid = np.vstack((top, bottom))

    out.write(grid)
    cv2.imshow("Multi-Video Analysis", grid)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ─── CLEANUP ───────────────────────────────────────────────────────
for cap in caps:
    cap.release()
out.release()
cv2.destroyAllWindows()

# ─── FINAL STATS & CSV EXPORT ──────────────────────────────────────
csv_rows = []
for i in range(4):
    total = len(unique_ids[i])
    avg = np.mean(speed_samples[i]) if speed_samples[i] else 0
    green_time = max(10, min(60, int((total / (avg + 1)) * 5)))
    print(f"Video {i + 1}: Vehicles: {total}, Avg Speed: {avg:.2f} m/s, Green Time: {green_time} s")
    for cls_id, cnt in classwise_counts[i].items():
        csv_rows.append({
            "Video": f"Video {i+1}",
            "Class": class_map.get(cls_id, "auto"),
            "Count": cnt
        })

df = pd.DataFrame(csv_rows)
df.to_csv("classwise_stats.csv", index=False)

# ─── APPEND FINAL OVERLAY FRAME (3 seconds) ────────────────────────
final_frame = np.zeros((out_h, out_w, 3), dtype=np.uint8)
for i in range(4):
    total = len(unique_ids[i])
    avg = np.mean(speed_samples[i]) if speed_samples[i] else 0
    green_time = max(10, min(60, int((total / (avg + 1)) * 5)))
    text = f"Video {i+1} → Count: {total}, Avg Speed: {avg:.1f} m/s, Green Time: {green_time} s"
    y = 40 + 60 * i
    cv2.putText(final_frame, text, (20, y), font, 1.0, (255, 255, 255), 2)

for _ in range(fps * 3):
    out = cv2.VideoWriter("output_grid.mp4", fourcc, fps, (out_w, out_h)) if False else None
    # Re-open writer in append mode is not supported; instead write to a new file or handle externally.
    # For simplicity, you can display this final frame here:
    cv2.imshow("Final Stats", final_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()



0: 480x640 7 cars, 4 motorcycles, 11.4ms
1: 480x640 4 cars, 2 motorcycles, 11.4ms
2: 480x640 3 cars, 11.4ms
3: 480x640 1 car, 11.4ms
Speed: 3.2ms preprocess, 11.4ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 cars, 3 motorcycles, 6.9ms
1: 480x640 4 cars, 3 motorcycles, 6.9ms
2: 480x640 1 car, 6.9ms
3: 480x640 3 cars, 3 motorcycles, 1 bus, 1 truck, 6.9ms
Speed: 2.8ms preprocess, 6.9ms inference, 3.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 7.0ms
1: 480x640 3 cars, 3 motorcycles, 7.0ms
2: 480x640 1 car, 7.0ms
3: 480x640 5 cars, 3 motorcycles, 1 bus, 1 truck, 7.0ms
Speed: 3.2ms preprocess, 7.0ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 cars, 1 motorcycle, 7.0ms
1: 480x640 3 cars, 3 motorcycles, 7.0ms
2: 480x640 1 car, 7.0ms
3: 480x640 4 cars, 3 motorcycles, 1 bus, 1 truck, 7.0ms
Speed: 2.8ms preprocess, 7.0ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 c

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
import pandas as pd
import time

# ─── CONFIGURATION ──────────────────────────────────────────────
video_paths = [
    "videos/video1.mp4",
    "videos/video2.mp4",
    "videos/video3.mp4",
    "videos/video4.mp4"
]
model_path = "runs/detect/train4/weights/best.pt"
# Adjusted conversion factor so speeds are more realistic (meters per pixel)
meters_per_pixel = 0.02  
INFER_IMG_SIZE = 640

font = cv2.FONT_HERSHEY_SIMPLEX
colors = {
    0: (0, 255, 0),     # car
    1: (0, 0, 255),     # motorcycle
    2: (255, 0, 0),     # bus
    3: (0, 255, 255),   # truck
    4: (255, 0, 255)    # auto
}
class_map = {
    0: "car",
    1: "motorcycle",
    2: "bus",
    3: "truck",
    4: "auto"
}
# ───────────────────────────────────────────────────────────────────

# ─── INITIALIZATION ────────────────────────────────────────────────
caps = [cv2.VideoCapture(p) for p in video_paths]
assert all(c.isOpened() for c in caps), "One or more videos failed to open."

model = YOLO(model_path)
fps = int(caps[0].get(cv2.CAP_PROP_FPS))
frame_w = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))

SUB_W = 480
SUB_H = int(frame_h * (SUB_W / frame_w))

out_w, out_h = SUB_W * 2, SUB_H * 2
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output_grid.mp4", fourcc, fps, (out_w, out_h))

cv2.namedWindow("Multi-Video Analysis", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Multi-Video Analysis", out_w, out_h)

unique_ids = [set() for _ in range(4)]
last_centroids = [dict() for _ in range(4)]
speed_samples = [list() for _ in range(4)]
classwise_counts = [defaultdict(int) for _ in range(4)]
# ───────────────────────────────────────────────────────────────────

# ─── MAIN PROCESSING LOOP ──────────────────────────────────────────
while True:
    frames = []
    rets = []
    for cap in caps:
        ret, frame = cap.read()
        rets.append(ret)
        frames.append(frame if ret else np.zeros((frame_h, frame_w, 3), dtype=np.uint8))
    if not any(rets):
        break  # exit if all videos ended

    results = model.track(
        frames,
        imgsz=INFER_IMG_SIZE,
        conf=0.25,
        iou=0.45,
        persist=True
    )
    processed_subframes = []

    for idx, det in enumerate(results):
        sub = cv2.resize(frames[idx], (SUB_W, SUB_H))
        if det.boxes is not None and len(det.boxes) > 0:
            xyxy = det.boxes.xyxy.cpu().numpy()               # (N,4)
            cls_ids = det.boxes.cls.cpu().numpy().astype(int) # (N,)
            track_ids = det.boxes.id.cpu().numpy().astype(int) if det.boxes.id is not None else [-1] * len(cls_ids)

            for (x1, y1, x2, y2), class_idx, track_id in zip(xyxy, cls_ids, track_ids):
                class_idx = class_idx if class_idx in class_map else 4  # remap unknown to 'auto'
                cx = int((x1 + x2) / 2)
                cy = int((y1 + y2) / 2)

                scale_x = SUB_W / frame_w
                scale_y = SUB_H / frame_h
                sx1 = int(x1 * scale_x)
                sy1 = int(y1 * scale_y)
                sx2 = int(x2 * scale_x)
                sy2 = int(y2 * scale_y)
                scx = int(cx * scale_x)
                scy = int(cy * scale_y)

                color = colors[class_idx]
                cv2.rectangle(sub, (sx1, sy1), (sx2, sy2), color, 2)
                cv2.putText(sub, f"{track_id}:{class_idx}", (sx1, sy1 - 5), font, 0.5, color, 1)

                unique_ids[idx].add(track_id)
                classwise_counts[idx][class_idx] += 1

                prev = last_centroids[idx].get(track_id, None)
                if prev is not None:
                    dx = cx - prev[0]
                    dy = cy - prev[1]
                    pixel_dist = np.hypot(dx, dy)
                    # speed in m/s
                    speed_m_s = pixel_dist * meters_per_pixel * fps
                    # convert to km/h
                    speed_kmh = speed_m_s * 3.6
                    speed_samples[idx].append(speed_kmh)
                    cv2.putText(sub, f"{speed_kmh:.1f} km/h", (sx1, sy2 + 12), font, 0.5, color, 1)

                last_centroids[idx][track_id] = (cx, cy)

        total = len(unique_ids[idx])
        avg_speed = np.mean(speed_samples[idx]) if speed_samples[idx] else 0
        # green time = 10 + 0.2 * total vehicles, capped at 60 seconds
        green_time = min(60, 10 + int(total * 0.2))

        stats = [
            f"Video {idx+1}",
            f"Count: {total}",
            f"Avg Speed: {avg_speed:.1f} km/h",
            f"Green Time: {green_time} s"
        ]
        y_offset = 20
        for stat in stats:
            cv2.putText(sub, stat, (10, y_offset), font, 0.6, (0, 0, 0), 3)
            cv2.putText(sub, stat, (10, y_offset), font, 0.6, (255, 255, 255), 1)
            y_offset += 25

        processed_subframes.append(sub)

    top = np.hstack((processed_subframes[0], processed_subframes[1]))
    bottom = np.hstack((processed_subframes[2], processed_subframes[3]))
    grid = np.vstack((top, bottom))

    out.write(grid)
    cv2.imshow("Multi-Video Analysis", grid)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ─── CLEANUP ───────────────────────────────────────────────────────
for cap in caps:
    cap.release()
out.release()
cv2.destroyAllWindows()

# ─── FINAL STATS & CSV EXPORT ──────────────────────────────────────
csv_rows = []
for i in range(4):
    total = len(unique_ids[i])
    avg = np.mean(speed_samples[i]) if speed_samples[i] else 0
    green_time = min(60, 10 + int(total * 0.2))
    print(f"Video {i + 1}: Vehicles: {total}, Avg Speed: {avg:.2f} km/h, Green Time: {green_time} s")
    for cls_id, cnt in classwise_counts[i].items():
        csv_rows.append({
            "Video": f"Video {i+1}",
            "Class": class_map.get(cls_id, "auto"),
            "Count": cnt
        })

df = pd.DataFrame(csv_rows)
df.to_csv("classwise_stats.csv", index=False)

# ─── APPEND FINAL OVERLAY FRAME (3 seconds) ────────────────────────
final_frame = np.zeros((out_h, out_w, 3), dtype=np.uint8)
for i in range(4):
    total = len(unique_ids[i])
    avg = np.mean(speed_samples[i]) if speed_samples[i] else 0
    green_time = min(60, 10 + int(total * 0.2))
    text = f"Video {i+1} → Count: {total}, Avg Speed: {avg:.1f} km/h, Green Time: {green_time} s"
    y = 40 + 60 * i
    cv2.putText(final_frame, text, (20, y), font, 1.0, (255, 255, 255), 2)

for _ in range(fps * 3):
    out.write(final_frame)



0: 480x640 7 cars, 4 motorcycles, 7.4ms
1: 480x640 4 cars, 2 motorcycles, 7.4ms
2: 480x640 3 cars, 7.4ms
3: 480x640 1 car, 7.4ms
Speed: 2.9ms preprocess, 7.4ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 cars, 3 motorcycles, 6.9ms
1: 480x640 4 cars, 3 motorcycles, 6.9ms
2: 480x640 1 car, 6.9ms
3: 480x640 3 cars, 3 motorcycles, 1 bus, 1 truck, 6.9ms
Speed: 2.9ms preprocess, 6.9ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 6.9ms
1: 480x640 3 cars, 3 motorcycles, 6.9ms
2: 480x640 1 car, 6.9ms
3: 480x640 5 cars, 3 motorcycles, 1 bus, 1 truck, 6.9ms
Speed: 2.7ms preprocess, 6.9ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 cars, 1 motorcycle, 6.9ms
1: 480x640 3 cars, 3 motorcycles, 6.9ms
2: 480x640 1 car, 6.9ms
3: 480x640 4 cars, 3 motorcycles, 1 bus, 1 truck, 6.9ms
Speed: 2.7ms preprocess, 6.9ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 cars, 

In [4]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
import pandas as pd
import time

# ─── CONFIGURATION ──────────────────────────────────────────────
video_paths = [
    "videos/video1.mp4",
    "videos/video2.mp4",
    "videos/video3.mp4",
    "videos/video4.mp4"
]
model_path = "runs/detect/train4/weights/best.pt"
meters_per_pixel = 0.01  # Reduced to yield realistic speeds (~30-40 km/h)
INFER_IMG_SIZE = 640

font = cv2.FONT_HERSHEY_SIMPLEX
colors = {
    0: (0, 255, 0),     # car
    1: (0, 0, 255),     # motorcycle
    2: (255, 0, 0),     # bus
    3: (0, 255, 255),   # truck
    4: (255, 0, 255)    # auto
}
class_map = {
    0: "car",
    1: "motorcycle",
    2: "bus",
    3: "truck",
    4: "auto"
}
# ───────────────────────────────────────────────────────────────────

# ─── INITIALIZATION ────────────────────────────────────────────────
caps = [cv2.VideoCapture(p) for p in video_paths]
assert all(c.isOpened() for c in caps), "One or more videos failed to open."

model = YOLO(model_path)
fps = int(caps[0].get(cv2.CAP_PROP_FPS))
frame_w = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))

SUB_W = 480
SUB_H = int(frame_h * (SUB_W / frame_w))

out_w, out_h = SUB_W * 2, SUB_H * 2
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output_grid2.mp4", fourcc, fps, (out_w, out_h))

cv2.namedWindow("Multi-Video Analysis", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Multi-Video Analysis", out_w, out_h)

unique_ids = [set() for _ in range(4)]
last_centroids = [dict() for _ in range(4)]
speed_samples = [list() for _ in range(4)]
classwise_counts = [defaultdict(int) for _ in range(4)]
# ───────────────────────────────────────────────────────────────────

# ─── MAIN PROCESSING LOOP ──────────────────────────────────────────
while True:
    frames = []
    rets = []
    for cap in caps:
        ret, frame = cap.read()
        rets.append(ret)
        frames.append(frame if ret else np.zeros((frame_h, frame_w, 3), dtype=np.uint8))
    if not any(rets):
        break  # exit if all videos ended

    results = model.track(
        frames,
        imgsz=INFER_IMG_SIZE,
        conf=0.25,
        iou=0.45,
        persist=True
    )
    processed_subframes = []

    for idx, det in enumerate(results):
        sub = cv2.resize(frames[idx], (SUB_W, SUB_H))
        if det.boxes is not None and len(det.boxes) > 0:
            xyxy = det.boxes.xyxy.cpu().numpy()               # (N,4)
            cls_ids = det.boxes.cls.cpu().numpy().astype(int) # (N,)
            track_ids = det.boxes.id.cpu().numpy().astype(int) if det.boxes.id is not None else [-1] * len(cls_ids)

            for (x1, y1, x2, y2), class_idx, track_id in zip(xyxy, cls_ids, track_ids):
                class_idx = class_idx if class_idx in class_map else 4  # remap unknown to 'auto'
                cx = int((x1 + x2) / 2)
                cy = int((y1 + y2) / 2)

                scale_x = SUB_W / frame_w
                scale_y = SUB_H / frame_h
                sx1 = int(x1 * scale_x)
                sy1 = int(y1 * scale_y)
                sx2 = int(x2 * scale_x)
                sy2 = int(y2 * scale_y)
                scx = int(cx * scale_x)
                scy = int(cy * scale_y)

                color = colors[class_idx]
                cv2.rectangle(sub, (sx1, sy1), (sx2, sy2), color, 2)
                label_text = f"{track_id}:{class_map[class_idx]}"
                cv2.putText(sub, label_text, (sx1, sy1 - 5), font, 0.5, color, 1)

                unique_ids[idx].add(track_id)
                classwise_counts[idx][class_idx] += 1

                prev = last_centroids[idx].get(track_id, None)
                if prev is not None:
                    dx = cx - prev[0]
                    dy = cy - prev[1]
                    pixel_dist = np.hypot(dx, dy)
                    # speed in m/s
                    speed_m_s = pixel_dist * meters_per_pixel * fps
                    # convert to km/h
                    speed_kmh = speed_m_s * 3.6
                    speed_samples[idx].append(speed_kmh)
                    cv2.putText(sub, f"{speed_kmh:.1f} km/h", (sx1, sy2 + 12), font, 0.5, color, 1)

                last_centroids[idx][track_id] = (cx, cy)

        total = len(unique_ids[idx])
        avg_speed = np.mean(speed_samples[idx]) if speed_samples[idx] else 0
        # green time = 10 + 0.2 * total vehicles, capped at 60 seconds
        green_time = min(60, 10 + int(total * 0.2))

        stats = [
            f"Video {idx+1}",
            f"Count: {total}",
            f"Avg Speed: {avg_speed:.1f} km/h",
            f"Green Time: {green_time} s"
        ]
        y_offset = 20
        for stat in stats:
            cv2.putText(sub, stat, (10, y_offset), font, 0.6, (0, 0, 0), 3)
            cv2.putText(sub, stat, (10, y_offset), font, 0.6, (255, 255, 255), 1)
            y_offset += 25

        processed_subframes.append(sub)

    top = np.hstack((processed_subframes[0], processed_subframes[1]))
    bottom = np.hstack((processed_subframes[2], processed_subframes[3]))
    grid = np.vstack((top, bottom))

    out.write(grid)
    cv2.imshow("Multi-Video Analysis", grid)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ─── CLEANUP ───────────────────────────────────────────────────────
for cap in caps:
    cap.release()
out.release()
cv2.destroyAllWindows()

# ─── FINAL STATS & CSV EXPORT ──────────────────────────────────────
csv_rows = []
for i in range(4):
    total = len(unique_ids[i])
    avg = np.mean(speed_samples[i]) if speed_samples[i] else 0
    green_time = min(60, 10 + int(total * 0.2))
    print(f"Video {i + 1}: Vehicles: {total}, Avg Speed: {avg:.2f} km/h, Green Time: {green_time} s")
    for cls_id, cnt in classwise_counts[i].items():
        csv_rows.append({
            "Video": f"Video {i+1}",
            "Class": class_map.get(cls_id, "auto"),
            "Count": cnt
        })

df = pd.DataFrame(csv_rows)
df.to_csv("classwise_stats.csv", index=False)

# ─── APPEND FINAL OVERLAY FRAME (3 seconds) ────────────────────────
final_frame = np.zeros((out_h, out_w, 3), dtype=np.uint8)
for i in range(4):
    total = len(unique_ids[i])
    avg = np.mean(speed_samples[i]) if speed_samples[i] else 0
    green_time = min(60, 10 + int(total * 0.2))
    text = f"Video {i+1} → Count: {total}, Avg Speed: {avg:.1f} km/h, Green Time: {green_time} s"
    y = 40 + 60 * i
    cv2.putText(final_frame, text, (20, y), font, 1.0, (255, 255, 255), 2)

for _ in range(fps * 3):
    out.write(final_frame)



0: 480x640 7 cars, 4 motorcycles, 29.8ms
1: 480x640 4 cars, 2 motorcycles, 29.8ms
2: 480x640 3 cars, 29.8ms
3: 480x640 1 car, 29.8ms
Speed: 5.2ms preprocess, 29.8ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 cars, 3 motorcycles, 6.9ms
1: 480x640 4 cars, 3 motorcycles, 6.9ms
2: 480x640 1 car, 6.9ms
3: 480x640 3 cars, 3 motorcycles, 1 bus, 1 truck, 6.9ms
Speed: 2.7ms preprocess, 6.9ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 6.9ms
1: 480x640 3 cars, 3 motorcycles, 6.9ms
2: 480x640 1 car, 6.9ms
3: 480x640 5 cars, 3 motorcycles, 1 bus, 1 truck, 6.9ms
Speed: 2.8ms preprocess, 6.9ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 cars, 1 motorcycle, 6.9ms
1: 480x640 3 cars, 3 motorcycles, 6.9ms
2: 480x640 1 car, 6.9ms
3: 480x640 4 cars, 3 motorcycles, 1 bus, 1 truck, 6.9ms
Speed: 2.7ms preprocess, 6.9ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 c

In [6]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
import pandas as pd
import time

# ─── CONFIGURATION ──────────────────────────────────────────────
video_paths = [
    "videos/video1.mp4",
    "videos/video2.mp4",
    "videos/video3.mp4",
    "videos/video4.mp4"
]
model_path = "runs/detect/train4/weights/best.pt"  # updated to train6
meters_per_pixel = 0.01  # same as before
INFER_IMG_SIZE = 640

font = cv2.FONT_HERSHEY_SIMPLEX
colors = {
    0: (0, 255, 0),     # car
    1: (0, 0, 255),     # motorcycle
    2: (255, 0, 0),     # bus
    3: (0, 255, 255),   # truck
    4: (255, 0, 255)    # auto
}
class_map = {
    0: "car",
    1: "motorcycle",
    2: "bus",
    3: "auto",     
    4: "auto"
}
# ───────────────────────────────────────────────────────────────────

# ─── INITIALIZATION ────────────────────────────────────────────────
caps = [cv2.VideoCapture(p) for p in video_paths]
assert all(c.isOpened() for c in caps), "One or more videos failed to open."

model = YOLO(model_path)
fps = int(caps[0].get(cv2.CAP_PROP_FPS))
frame_w = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))

SUB_W = 480
SUB_H = int(frame_h * (SUB_W / frame_w))

out_w, out_h = SUB_W * 2, SUB_H * 2
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output_grid3.mp4", fourcc, fps, (out_w, out_h))

cv2.namedWindow("Multi-Video Analysis", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Multi-Video Analysis", out_w, out_h)

unique_ids = [set() for _ in range(4)]
last_centroids = [dict() for _ in range(4)]
speed_samples = [list() for _ in range(4)]
classwise_counts = [defaultdict(int) for _ in range(4)]
# ───────────────────────────────────────────────────────────────────

# ─── MAIN PROCESSING LOOP ──────────────────────────────────────────
while True:
    frames = []
    rets = []
    for cap in caps:
        ret, frame = cap.read()
        rets.append(ret)
        frames.append(frame if ret else np.zeros((frame_h, frame_w, 3), dtype=np.uint8))
    if not any(rets):
        break  # exit if all videos ended

    results = model.track(
        frames,
        imgsz=INFER_IMG_SIZE,
        conf=0.25,
        iou=0.45,
        persist=True
    )
    processed_subframes = []

    for idx, det in enumerate(results):
        sub = cv2.resize(frames[idx], (SUB_W, SUB_H))
        if det.boxes is not None and len(det.boxes) > 0:
            xyxy = det.boxes.xyxy.cpu().numpy()               # (N,4)
            cls_ids = det.boxes.cls.cpu().numpy().astype(int) # (N,)
            track_ids = det.boxes.id.cpu().numpy().astype(int) if det.boxes.id is not None else [-1] * len(cls_ids)

            for (x1, y1, x2, y2), class_idx, track_id in zip(xyxy, cls_ids, track_ids):
                # Remap class 3 (truck) to auto (4)
                if class_idx == 3:
                    class_idx = 4
                cx = int((x1 + x2) / 2)
                cy = int((y1 + y2) / 2)

                scale_x = SUB_W / frame_w
                scale_y = SUB_H / frame_h
                sx1 = int(x1 * scale_x)
                sy1 = int(y1 * scale_y)
                sx2 = int(x2 * scale_x)
                sy2 = int(y2 * scale_y)
                scx = int(cx * scale_x)
                scy = int(cy * scale_y)

                color = colors[class_idx]
                cv2.rectangle(sub, (sx1, sy1), (sx2, sy2), color, 2)
                label_text = f"{track_id}:{class_map[class_idx]}"
                cv2.putText(sub, label_text, (sx1, sy1 - 5), font, 0.5, color, 1)

                unique_ids[idx].add(track_id)
                classwise_counts[idx][class_idx] += 1

                prev = last_centroids[idx].get(track_id, None)
                if prev is not None:
                    dx = cx - prev[0]
                    dy = cy - prev[1]
                    pixel_dist = np.hypot(dx, dy)
                    # speed in m/s
                    speed_m_s = pixel_dist * meters_per_pixel * fps
                    # convert to km/h and then divide by 6
                    speed_kmh = (speed_m_s * 3.6) / 6
                    speed_samples[idx].append(speed_kmh)
                    cv2.putText(sub, f"{speed_kmh:.1f} km/h", (sx1, sy2 + 12), font, 0.5, color, 1)

                last_centroids[idx][track_id] = (cx, cy)

        total = len(unique_ids[idx])
        avg_speed = np.mean(speed_samples[idx]) if speed_samples[idx] else 0
        # green time = 10 + 0.2 * total vehicles, capped at 60 seconds
        green_time = min(60, 10 + int(total * 0.2))

        stats = [
            f"Video {idx+1}",
            f"Count: {total}",
            f"Avg Speed: {avg_speed:.1f} km/h",
            f"Green Time: {green_time} s"
        ]
        y_offset = 20
        for stat in stats:
            cv2.putText(sub, stat, (10, y_offset), font, 0.6, (0, 0, 0), 3)
            cv2.putText(sub, stat, (10, y_offset), font, 0.6, (255, 255, 255), 1)
            y_offset += 25

        processed_subframes.append(sub)

    top = np.hstack((processed_subframes[0], processed_subframes[1]))
    bottom = np.hstack((processed_subframes[2], processed_subframes[3]))
    grid = np.vstack((top, bottom))

    out.write(grid)
    cv2.imshow("Multi-Video Analysis", grid)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ─── CLEANUP ───────────────────────────────────────────────────────
for cap in caps:
    cap.release()
out.release()
cv2.destroyAllWindows()

# ─── FINAL STATS & CSV EXPORT ──────────────────────────────────────
csv_rows = []
for i in range(4):
    total = len(unique_ids[i])
    avg = np.mean(speed_samples[i]) if speed_samples[i] else 0
    green_time = min(60, 10 + int(total * 0.2))
    print(f"Video {i + 1}: Vehicles: {total}, Avg Speed: {avg:.2f} km/h, Green Time: {green_time} s")
    for cls_id, cnt in classwise_counts[i].items():
        csv_rows.append({
            "Video": f"Video {i+1}",
            "Class": class_map.get(cls_id, "auto"),
            "Count": cnt
        })

df = pd.DataFrame(csv_rows)
df.to_csv("classwise_stats.csv", index=False)

# ─── APPEND FINAL OVERLAY FRAME (3 seconds) ────────────────────────
final_frame = np.zeros((out_h, out_w, 3), dtype=np.uint8)
for i in range(4):
    total = len(unique_ids[i])
    avg = np.mean(speed_samples[i]) if speed_samples[i] else 0
    green_time = min(60, 10 + int(total * 0.2))
    text = f"Video {i+1} → Count: {total}, Avg Speed: {avg:.1f} km/h, Green Time: {green_time} s"
    y = 40 + 60 * i
    cv2.putText(final_frame, text, (20, y), font, 1.0, (255, 255, 255), 2)

for _ in range(fps * 3):
    out.write(final_frame)



0: 480x640 7 cars, 4 motorcycles, 36.8ms
1: 480x640 4 cars, 2 motorcycles, 36.8ms
2: 480x640 3 cars, 36.8ms
3: 480x640 1 car, 36.8ms
Speed: 5.0ms preprocess, 36.8ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 cars, 3 motorcycles, 6.9ms
1: 480x640 4 cars, 3 motorcycles, 6.9ms
2: 480x640 1 car, 6.9ms
3: 480x640 3 cars, 3 motorcycles, 1 bus, 1 truck, 6.9ms
Speed: 2.8ms preprocess, 6.9ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 6.9ms
1: 480x640 3 cars, 3 motorcycles, 6.9ms
2: 480x640 1 car, 6.9ms
3: 480x640 5 cars, 3 motorcycles, 1 bus, 1 truck, 6.9ms
Speed: 2.7ms preprocess, 6.9ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 cars, 1 motorcycle, 7.0ms
1: 480x640 3 cars, 3 motorcycles, 7.0ms
2: 480x640 1 car, 7.0ms
3: 480x640 4 cars, 3 motorcycles, 1 bus, 1 truck, 7.0ms
Speed: 2.9ms preprocess, 7.0ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 c

In [7]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
import pandas as pd
import time

# ─── CONFIGURATION ──────────────────────────────────────────────
video_paths = [
    "videos/video5.mp4",
    "videos/video6.mp4",
    "videos/video7.mp4",
    "videos/video8.mp4"
]
model_path = "runs/detect/train4/weights/best.pt"  # updated to train6
meters_per_pixel = 0.01  # same as before
INFER_IMG_SIZE = 640

font = cv2.FONT_HERSHEY_SIMPLEX
colors = {
    0: (0, 255, 0),     # car
    1: (0, 0, 255),     # motorcycle
    2: (255, 0, 0),     # bus
    3: (0, 255, 255),   # truck
    4: (255, 0, 255)    # auto
}
class_map = {
    0: "car",
    1: "motorcycle",
    2: "bus",
    3: "auto",     
    4: "auto"
}
# ───────────────────────────────────────────────────────────────────

# ─── INITIALIZATION ────────────────────────────────────────────────
caps = [cv2.VideoCapture(p) for p in video_paths]
assert all(c.isOpened() for c in caps), "One or more videos failed to open."

model = YOLO(model_path)
fps = int(caps[0].get(cv2.CAP_PROP_FPS))
frame_w = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))

SUB_W = 480
SUB_H = int(frame_h * (SUB_W / frame_w))

out_w, out_h = SUB_W * 2, SUB_H * 2
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output_grid4.mp4", fourcc, fps, (out_w, out_h))

cv2.namedWindow("Multi-Video Analysis", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Multi-Video Analysis", out_w, out_h)

unique_ids = [set() for _ in range(4)]
last_centroids = [dict() for _ in range(4)]
speed_samples = [list() for _ in range(4)]
classwise_counts = [defaultdict(int) for _ in range(4)]
# ───────────────────────────────────────────────────────────────────

# ─── MAIN PROCESSING LOOP ──────────────────────────────────────────
while True:
    frames = []
    rets = []
    for cap in caps:
        ret, frame = cap.read()
        rets.append(ret)
        frames.append(frame if ret else np.zeros((frame_h, frame_w, 3), dtype=np.uint8))
    if not any(rets):
        break  # exit if all videos ended

    results = model.track(
        frames,
        imgsz=INFER_IMG_SIZE,
        conf=0.25,
        iou=0.45,
        persist=True
    )
    processed_subframes = []

    for idx, det in enumerate(results):
        sub = cv2.resize(frames[idx], (SUB_W, SUB_H))
        if det.boxes is not None and len(det.boxes) > 0:
            xyxy = det.boxes.xyxy.cpu().numpy()               # (N,4)
            cls_ids = det.boxes.cls.cpu().numpy().astype(int) # (N,)
            track_ids = det.boxes.id.cpu().numpy().astype(int) if det.boxes.id is not None else [-1] * len(cls_ids)

            for (x1, y1, x2, y2), class_idx, track_id in zip(xyxy, cls_ids, track_ids):
                # Remap class 3 (truck) to auto (4)
                if class_idx == 3:
                    class_idx = 4
                cx = int((x1 + x2) / 2)
                cy = int((y1 + y2) / 2)

                scale_x = SUB_W / frame_w
                scale_y = SUB_H / frame_h
                sx1 = int(x1 * scale_x)
                sy1 = int(y1 * scale_y)
                sx2 = int(x2 * scale_x)
                sy2 = int(y2 * scale_y)
                scx = int(cx * scale_x)
                scy = int(cy * scale_y)

                color = colors[class_idx]
                cv2.rectangle(sub, (sx1, sy1), (sx2, sy2), color, 2)
                label_text = f"{track_id}:{class_map[class_idx]}"
                cv2.putText(sub, label_text, (sx1, sy1 - 5), font, 0.5, color, 1)

                unique_ids[idx].add(track_id)
                classwise_counts[idx][class_idx] += 1

                prev = last_centroids[idx].get(track_id, None)
                if prev is not None:
                    dx = cx - prev[0]
                    dy = cy - prev[1]
                    pixel_dist = np.hypot(dx, dy)
                    # speed in m/s
                    speed_m_s = pixel_dist * meters_per_pixel * fps
                    # convert to km/h and then divide by 6
                    speed_kmh = (speed_m_s * 3.6) / 6
                    speed_samples[idx].append(speed_kmh)
                    cv2.putText(sub, f"{speed_kmh:.1f} km/h", (sx1, sy2 + 12), font, 0.5, color, 1)

                last_centroids[idx][track_id] = (cx, cy)

        total = len(unique_ids[idx])
        avg_speed = np.mean(speed_samples[idx]) if speed_samples[idx] else 0
        # green time = 10 + 0.2 * total vehicles, capped at 60 seconds
        green_time = min(60, 10 + int(total * 0.2))

        stats = [
            f"Video {idx+1}",
            f"Count: {total}",
            f"Avg Speed: {avg_speed:.1f} km/h",
            f"Green Time: {green_time} s"
        ]
        y_offset = 20
        for stat in stats:
            cv2.putText(sub, stat, (10, y_offset), font, 0.6, (0, 0, 0), 3)
            cv2.putText(sub, stat, (10, y_offset), font, 0.6, (255, 255, 255), 1)
            y_offset += 25

        processed_subframes.append(sub)

    top = np.hstack((processed_subframes[0], processed_subframes[1]))
    bottom = np.hstack((processed_subframes[2], processed_subframes[3]))
    grid = np.vstack((top, bottom))

    out.write(grid)
    cv2.imshow("Multi-Video Analysis", grid)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ─── CLEANUP ───────────────────────────────────────────────────────
for cap in caps:
    cap.release()
out.release()
cv2.destroyAllWindows()

# ─── FINAL STATS & CSV EXPORT ──────────────────────────────────────
csv_rows = []
for i in range(4):
    total = len(unique_ids[i])
    avg = np.mean(speed_samples[i]) if speed_samples[i] else 0
    green_time = min(60, 10 + int(total * 0.2))
    print(f"Video {i + 1}: Vehicles: {total}, Avg Speed: {avg:.2f} km/h, Green Time: {green_time} s")
    for cls_id, cnt in classwise_counts[i].items():
        csv_rows.append({
            "Video": f"Video {i+1}",
            "Class": class_map.get(cls_id, "auto"),
            "Count": cnt
        })

df = pd.DataFrame(csv_rows)
df.to_csv("classwise_stats.csv", index=False)

# ─── APPEND FINAL OVERLAY FRAME (3 seconds) ────────────────────────
final_frame = np.zeros((out_h, out_w, 3), dtype=np.uint8)
for i in range(4):
    total = len(unique_ids[i])
    avg = np.mean(speed_samples[i]) if speed_samples[i] else 0
    green_time = min(60, 10 + int(total * 0.2))
    text = f"Video {i+1} → Count: {total}, Avg Speed: {avg:.1f} km/h, Green Time: {green_time} s"
    y = 40 + 60 * i
    cv2.putText(final_frame, text, (20, y), font, 1.0, (255, 255, 255), 2)

for _ in range(fps * 3):
    out.write(final_frame)



0: 384x640 5 cars, 1 motorcycle, 30.4ms
1: 384x640 4 cars, 2 trucks, 30.4ms
2: 384x640 6 cars, 1 truck, 30.4ms
3: 384x640 4 cars, 30.4ms
Speed: 4.3ms preprocess, 30.4ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 motorcycle, 5.7ms
1: 384x640 4 cars, 2 trucks, 5.7ms
2: 384x640 6 cars, 1 truck, 5.7ms
3: 384x640 5 cars, 5.7ms
Speed: 2.4ms preprocess, 5.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 motorcycle, 5.8ms
1: 384x640 4 cars, 2 trucks, 5.8ms
2: 384x640 8 cars, 1 motorcycle, 5.8ms
3: 384x640 5 cars, 5.8ms
Speed: 2.7ms preprocess, 5.8ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 motorcycle, 5.8ms
1: 384x640 5 cars, 2 trucks, 5.8ms
2: 384x640 6 cars, 1 motorcycle, 1 truck, 5.8ms
3: 384x640 4 cars, 5.8ms
Speed: 2.3ms preprocess, 5.8ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 2 motorcycles, 7.6ms
1: 384x640 5 ca

In [8]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
import pandas as pd
import time

# ─── CONFIGURATION ──────────────────────────────────────────────
video_paths = [
    "videos/video10.mp4",
    "videos/video11.mp4",
    "videos/video12.mp4",
    "videos/video13.mp4"
]
model_path = "runs/detect/train4/weights/best.pt"  # updated to train6
meters_per_pixel = 0.01  # same as before
INFER_IMG_SIZE = 640

font = cv2.FONT_HERSHEY_SIMPLEX
colors = {
    0: (0, 255, 0),     # car
    1: (0, 0, 255),     # motorcycle
    2: (255, 0, 0),     # bus
    3: (0, 255, 255),   # truck
    4: (255, 0, 255)    # auto
}
class_map = {
    0: "car",
    1: "motorcycle",
    2: "bus",
    3: "auto",     
    4: "auto"
}
# ───────────────────────────────────────────────────────────────────

# ─── INITIALIZATION ────────────────────────────────────────────────
caps = [cv2.VideoCapture(p) for p in video_paths]
assert all(c.isOpened() for c in caps), "One or more videos failed to open."

model = YOLO(model_path)
fps = int(caps[0].get(cv2.CAP_PROP_FPS))
frame_w = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))

SUB_W = 480
SUB_H = int(frame_h * (SUB_W / frame_w))

out_w, out_h = SUB_W * 2, SUB_H * 2
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output_grid5.mp4", fourcc, fps, (out_w, out_h))

cv2.namedWindow("Multi-Video Analysis", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Multi-Video Analysis", out_w, out_h)

unique_ids = [set() for _ in range(4)]
last_centroids = [dict() for _ in range(4)]
speed_samples = [list() for _ in range(4)]
classwise_counts = [defaultdict(int) for _ in range(4)]
# ───────────────────────────────────────────────────────────────────

# ─── MAIN PROCESSING LOOP ──────────────────────────────────────────
while True:
    frames = []
    rets = []
    for cap in caps:
        ret, frame = cap.read()
        rets.append(ret)
        frames.append(frame if ret else np.zeros((frame_h, frame_w, 3), dtype=np.uint8))
    if not any(rets):
        break  # exit if all videos ended

    results = model.track(
        frames,
        imgsz=INFER_IMG_SIZE,
        conf=0.25,
        iou=0.45,
        persist=True
    )
    processed_subframes = []

    for idx, det in enumerate(results):
        sub = cv2.resize(frames[idx], (SUB_W, SUB_H))
        if det.boxes is not None and len(det.boxes) > 0:
            xyxy = det.boxes.xyxy.cpu().numpy()               # (N,4)
            cls_ids = det.boxes.cls.cpu().numpy().astype(int) # (N,)
            track_ids = det.boxes.id.cpu().numpy().astype(int) if det.boxes.id is not None else [-1] * len(cls_ids)

            for (x1, y1, x2, y2), class_idx, track_id in zip(xyxy, cls_ids, track_ids):
                # Remap class 3 (truck) to auto (4)
                if class_idx == 3:
                    class_idx = 4
                cx = int((x1 + x2) / 2)
                cy = int((y1 + y2) / 2)

                scale_x = SUB_W / frame_w
                scale_y = SUB_H / frame_h
                sx1 = int(x1 * scale_x)
                sy1 = int(y1 * scale_y)
                sx2 = int(x2 * scale_x)
                sy2 = int(y2 * scale_y)
                scx = int(cx * scale_x)
                scy = int(cy * scale_y)

                color = colors[class_idx]
                cv2.rectangle(sub, (sx1, sy1), (sx2, sy2), color, 2)
                label_text = f"{track_id}:{class_map[class_idx]}"
                cv2.putText(sub, label_text, (sx1, sy1 - 5), font, 0.5, color, 1)

                unique_ids[idx].add(track_id)
                classwise_counts[idx][class_idx] += 1

                prev = last_centroids[idx].get(track_id, None)
                if prev is not None:
                    dx = cx - prev[0]
                    dy = cy - prev[1]
                    pixel_dist = np.hypot(dx, dy)
                    # speed in m/s
                    speed_m_s = pixel_dist * meters_per_pixel * fps
                    # convert to km/h and then divide by 6
                    speed_kmh = (speed_m_s * 3.6) / 6
                    speed_samples[idx].append(speed_kmh)
                    cv2.putText(sub, f"{speed_kmh:.1f} km/h", (sx1, sy2 + 12), font, 0.5, color, 1)

                last_centroids[idx][track_id] = (cx, cy)

        total = len(unique_ids[idx])
        avg_speed = np.mean(speed_samples[idx]) if speed_samples[idx] else 0
        # green time = 10 + 0.2 * total vehicles, capped at 60 seconds
        green_time = min(60, 10 + int(total * 0.2))

        stats = [
            f"Video {idx+1}",
            f"Count: {total}",
            f"Avg Speed: {avg_speed:.1f} km/h",
            f"Green Time: {green_time} s"
        ]
        y_offset = 20
        for stat in stats:
            cv2.putText(sub, stat, (10, y_offset), font, 0.6, (0, 0, 0), 3)
            cv2.putText(sub, stat, (10, y_offset), font, 0.6, (255, 255, 255), 1)
            y_offset += 25

        processed_subframes.append(sub)

    top = np.hstack((processed_subframes[0], processed_subframes[1]))
    bottom = np.hstack((processed_subframes[2], processed_subframes[3]))
    grid = np.vstack((top, bottom))

    out.write(grid)
    cv2.imshow("Multi-Video Analysis", grid)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ─── CLEANUP ───────────────────────────────────────────────────────
for cap in caps:
    cap.release()
out.release()
cv2.destroyAllWindows()

# ─── FINAL STATS & CSV EXPORT ──────────────────────────────────────
csv_rows = []
for i in range(4):
    total = len(unique_ids[i])
    avg = np.mean(speed_samples[i]) if speed_samples[i] else 0
    green_time = min(60, 10 + int(total * 0.2))
    print(f"Video {i + 1}: Vehicles: {total}, Avg Speed: {avg:.2f} km/h, Green Time: {green_time} s")
    for cls_id, cnt in classwise_counts[i].items():
        csv_rows.append({
            "Video": f"Video {i+1}",
            "Class": class_map.get(cls_id, "auto"),
            "Count": cnt
        })

df = pd.DataFrame(csv_rows)
df.to_csv("classwise_stats.csv", index=False)

# ─── APPEND FINAL OVERLAY FRAME (3 seconds) ────────────────────────
final_frame = np.zeros((out_h, out_w, 3), dtype=np.uint8)
for i in range(4):
    total = len(unique_ids[i])
    avg = np.mean(speed_samples[i]) if speed_samples[i] else 0
    green_time = min(60, 10 + int(total * 0.2))
    text = f"Video {i+1} → Count: {total}, Avg Speed: {avg:.1f} km/h, Green Time: {green_time} s"
    y = 40 + 60 * i
    cv2.putText(final_frame, text, (20, y), font, 1.0, (255, 255, 255), 2)

for _ in range(fps * 3):
    out.write(final_frame)



0: 384x640 6 cars, 1 truck, 17.2ms
1: 384x640 4 cars, 2 trucks, 17.2ms
2: 384x640 1 car, 17.2ms
3: 384x640 1 car, 17.2ms
Speed: 4.4ms preprocess, 17.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.8ms
1: 384x640 1 car, 5.8ms
2: 384x640 3 cars, 1 motorcycle, 5.8ms
3: 384x640 7 cars, 1 truck, 5.8ms
Speed: 2.8ms preprocess, 5.8ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.8ms
1: 384x640 4 cars, 2 trucks, 5.8ms
2: 384x640 4 cars, 1 motorcycle, 5.8ms
3: 384x640 2 cars, 5.8ms
Speed: 2.8ms preprocess, 5.8ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.8ms
1: 384x640 5 cars, 2 trucks, 5.8ms
2: 384x640 5 cars, 1 motorcycle, 5.8ms
3: 384x640 1 car, 5.8ms
Speed: 2.6ms preprocess, 5.8ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.8ms
1: 384x640 5 cars, 2 trucks, 5.8ms
2: 384x640 1 car, 5.8ms
3: 384x640 1 car, 5.8ms
Speed: 2.5ms

In [9]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
import pandas as pd
import time

# ─── CONFIGURATION ──────────────────────────────────────────────
video_paths = [
    "videos/video21.mp4",
    "videos/video22.mp4",
    "videos/video23.mp4",
    "videos/video24.mp4"
]
model_path = "runs/detect/train4/weights/best.pt"  # updated to train6
meters_per_pixel = 0.01  # same as before
INFER_IMG_SIZE = 640

font = cv2.FONT_HERSHEY_SIMPLEX
colors = {
    0: (0, 255, 0),     # car
    1: (0, 0, 255),     # motorcycle
    2: (255, 0, 0),     # bus
    3: (0, 255, 255),   # truck
    4: (255, 0, 255)    # auto
}
class_map = {
    0: "car",
    1: "motorcycle",
    2: "bus",
    3: "auto",     
    4: "auto"
}
# ───────────────────────────────────────────────────────────────────

# ─── INITIALIZATION ────────────────────────────────────────────────
caps = [cv2.VideoCapture(p) for p in video_paths]
assert all(c.isOpened() for c in caps), "One or more videos failed to open."

model = YOLO(model_path)
fps = int(caps[0].get(cv2.CAP_PROP_FPS))
frame_w = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))

SUB_W = 480
SUB_H = int(frame_h * (SUB_W / frame_w))

out_w, out_h = SUB_W * 2, SUB_H * 2
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output_grid7.mp4", fourcc, fps, (out_w, out_h))

cv2.namedWindow("Multi-Video Analysis", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Multi-Video Analysis", out_w, out_h)

unique_ids = [set() for _ in range(4)]
last_centroids = [dict() for _ in range(4)]
speed_samples = [list() for _ in range(4)]
classwise_counts = [defaultdict(int) for _ in range(4)]
# ───────────────────────────────────────────────────────────────────

# ─── MAIN PROCESSING LOOP ──────────────────────────────────────────
while True:
    frames = []
    rets = []
    for cap in caps:
        ret, frame = cap.read()
        rets.append(ret)
        frames.append(frame if ret else np.zeros((frame_h, frame_w, 3), dtype=np.uint8))
    if not any(rets):
        break  # exit if all videos ended

    results = model.track(
        frames,
        imgsz=INFER_IMG_SIZE,
        conf=0.25,
        iou=0.45,
        persist=True
    )
    processed_subframes = []

    for idx, det in enumerate(results):
        sub = cv2.resize(frames[idx], (SUB_W, SUB_H))
        if det.boxes is not None and len(det.boxes) > 0:
            xyxy = det.boxes.xyxy.cpu().numpy()               # (N,4)
            cls_ids = det.boxes.cls.cpu().numpy().astype(int) # (N,)
            track_ids = det.boxes.id.cpu().numpy().astype(int) if det.boxes.id is not None else [-1] * len(cls_ids)

            for (x1, y1, x2, y2), class_idx, track_id in zip(xyxy, cls_ids, track_ids):
                # Remap class 3 (truck) to auto (4)
                if class_idx == 3:
                    class_idx = 4
                cx = int((x1 + x2) / 2)
                cy = int((y1 + y2) / 2)

                scale_x = SUB_W / frame_w
                scale_y = SUB_H / frame_h
                sx1 = int(x1 * scale_x)
                sy1 = int(y1 * scale_y)
                sx2 = int(x2 * scale_x)
                sy2 = int(y2 * scale_y)
                scx = int(cx * scale_x)
                scy = int(cy * scale_y)

                color = colors[class_idx]
                cv2.rectangle(sub, (sx1, sy1), (sx2, sy2), color, 2)
                label_text = f"{track_id}:{class_map[class_idx]}"
                cv2.putText(sub, label_text, (sx1, sy1 - 5), font, 0.5, color, 1)

                unique_ids[idx].add(track_id)
                classwise_counts[idx][class_idx] += 1

                prev = last_centroids[idx].get(track_id, None)
                if prev is not None:
                    dx = cx - prev[0]
                    dy = cy - prev[1]
                    pixel_dist = np.hypot(dx, dy)
                    # speed in m/s
                    speed_m_s = pixel_dist * meters_per_pixel * fps
                    # convert to km/h and then divide by 6
                    speed_kmh = (speed_m_s * 3.6) / 6
                    speed_samples[idx].append(speed_kmh)
                    cv2.putText(sub, f"{speed_kmh:.1f} km/h", (sx1, sy2 + 12), font, 0.5, color, 1)

                last_centroids[idx][track_id] = (cx, cy)

        total = len(unique_ids[idx])
        avg_speed = np.mean(speed_samples[idx]) if speed_samples[idx] else 0
        # green time = 10 + 0.2 * total vehicles, capped at 60 seconds
        green_time = min(60, 10 + int(total * 0.2))

        stats = [
            f"Video {idx+1}",
            f"Count: {total}",
            f"Avg Speed: {avg_speed:.1f} km/h",
            f"Green Time: {green_time} s"
        ]
        y_offset = 20
        for stat in stats:
            cv2.putText(sub, stat, (10, y_offset), font, 0.6, (0, 0, 0), 3)
            cv2.putText(sub, stat, (10, y_offset), font, 0.6, (255, 255, 255), 1)
            y_offset += 25

        processed_subframes.append(sub)

    top = np.hstack((processed_subframes[0], processed_subframes[1]))
    bottom = np.hstack((processed_subframes[2], processed_subframes[3]))
    grid = np.vstack((top, bottom))

    out.write(grid)
    cv2.imshow("Multi-Video Analysis", grid)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ─── CLEANUP ───────────────────────────────────────────────────────
for cap in caps:
    cap.release()
out.release()
cv2.destroyAllWindows()

# ─── FINAL STATS & CSV EXPORT ──────────────────────────────────────
csv_rows = []
for i in range(4):
    total = len(unique_ids[i])
    avg = np.mean(speed_samples[i]) if speed_samples[i] else 0
    green_time = min(60, 10 + int(total * 0.2))
    print(f"Video {i + 1}: Vehicles: {total}, Avg Speed: {avg:.2f} km/h, Green Time: {green_time} s")
    for cls_id, cnt in classwise_counts[i].items():
        csv_rows.append({
            "Video": f"Video {i+1}",
            "Class": class_map.get(cls_id, "auto"),
            "Count": cnt
        })

df = pd.DataFrame(csv_rows)
df.to_csv("classwise_stats.csv", index=False)

# ─── APPEND FINAL OVERLAY FRAME (3 seconds) ────────────────────────
final_frame = np.zeros((out_h, out_w, 3), dtype=np.uint8)
for i in range(4):
    total = len(unique_ids[i])
    avg = np.mean(speed_samples[i]) if speed_samples[i] else 0
    green_time = min(60, 10 + int(total * 0.2))
    text = f"Video {i+1} → Count: {total}, Avg Speed: {avg:.1f} km/h, Green Time: {green_time} s"
    y = 40 + 60 * i
    cv2.putText(final_frame, text, (20, y), font, 1.0, (255, 255, 255), 2)

for _ in range(fps * 3):
    out.write(final_frame)



0: 384x640 8 cars, 1 motorcycle, 37.9ms
1: 384x640 4 cars, 2 motorcycles, 37.9ms
2: 384x640 2 cars, 37.9ms
3: 384x640 2 cars, 2 motorcycles, 2 trucks, 37.9ms
Speed: 4.7ms preprocess, 37.9ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 5.8ms
1: 384x640 4 cars, 1 motorcycle, 5.8ms
2: 384x640 1 car, 5.8ms
3: 384x640 3 cars, 2 motorcycles, 1 bus, 2 trucks, 5.8ms
Speed: 2.7ms preprocess, 5.8ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.8ms
1: 384x640 1 car, 5.8ms
2: 384x640 2 cars, 5.8ms
3: 384x640 1 car, 5.8ms
Speed: 2.4ms preprocess, 5.8ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 motorcycle, 5.8ms
1: 384x640 3 cars, 2 motorcycles, 5.8ms
2: 384x640 1 car, 5.8ms
3: 384x640 3 cars, 4 motorcycles, 1 bus, 2 trucks, 5.8ms
Speed: 2.5ms preprocess, 5.8ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 motorcycle, 5.8ms
1: 384x640